In [8]:
pip install pexpect
pip install rosbags

SyntaxError: invalid syntax (3299651973.py, line 1)

Lets collect the scan data closest to a user-defined timestamp value from the robot. For this, you need to extract data from the bag file.

In [ ]:
## Reading from bag file

from rosbags.rosbag2 import Reader
from rosbags.typesys import Stores, get_typestore

bag_file_path = "rosbags/bags/ros2bag_test_1.db3"



no_of_messages_to_read = 5
scan_list = list()

#Uniform Resource Identifier (uri) should be assigned to path of the folder in which .db3 file is saved
storage_options = StorageOptions(uri=bag_file_path, storage_id="sqlite3") 

# Common Data Representation (cdr) is the binary serialisation format used in ROS2, so read and write format is set to "cdr"
converter_options = ConverterOptions(
        input_serialization_format="cdr", output_serialization_format="cdr")

# To read messages in sequence, setting reader to SequentialReader
reader = SequentialReader()
reader.open(storage_options, converter_options)

# Set desired scan_topic_name name (string)
scan_topic_name = '/scan'
count=0

# Example to read topic, its data and the time at which it was published with the data is provided below.
# Here the first set of scan data points are stored.
while reader.has_next():
    
    if count < no_of_messages_to_read:
        # the timestamp 't' has units of nanoseconds. If you want to get data at particular time, please change code accordingly
        (topic, data, t) = reader.read_next()
        
        if topic == scan_topic_name:
            count += 1
            # deserialize_message is used to decode the message. It takes the message(data) and its type as input
            scan_data = deserialize_message(data, LaserScan())
            scan_list.append(scan_data)
    else:
        break
# Interesting Excercise: Please feel free to move around robot to get it inside the room in simulation. Using above 
# example, try to get the scan data when the robot is inside the room.
scan_list

# Interesting exercise (optional): Move the robot around to get it inside the room in the simulation. Using the above 
# example, try to get the scan data when the robot is inside the room.

ModuleNotFoundError: No module named 'rosbags'

In [9]:
print('idea')

idea


### You have to convert the scan data from polar coordinates to cartesian coordinates for better interpretation and integration with other functionalities. For this, you need to get the minimum and maximum angular range of the laser scanner and also the resolution of angles (which gives you the number of data points in a single scan).

#### Note: we need to figure out how can you get these details from scan_data. Also, it might interest you to know with respect to which frame the data was published.

In [ ]:
## Helper function to convert scan data from polar to Cartesian coordinates
def np_polar2cart(np_polar: np.ndarray):
    """

    Keyword arguments:
    np_polar: np.ndarray -- A 2D numpy array, where each row represents the distance to an obstacle at a 
                            corresponding angle in radians. Thus, the number of rows will be equal to
                            the number of scan data points

    Return:
    np_cart: np.ndarray -- A 2D numpy array, where each row consists of X and Y Cartesian coordinates
                           of an obstacle
    """
    
    # YOUR CODE HERE
    cartesian_data = list()
    for ro,theta in np_polar:
        if not isinf(ro):  # Ignore infinite values
            x = ro * cos(theta)
            y = ro * sin(theta)
            cartesian_data.append(np.array([x, y]))
    
    return np.array(cartesian_data)
    
cartesianList = list()


for scan_data in scan_list:
    angles = np.arange(scan_data.angle_min, scan_data.angle_max, scan_data.angle_increment)
    polar = list(zip(scan_data.ranges, angles))

    cartesian = np_polar2cart(polar)
    cartesianList.append(cartesian)
    # Printing only the valid mesurements
cartesianList   


Plot the points using the Cartesian coordinates from your above implementation.

Note: our plot will have a corresponding title, axis labels, grid, and legend.

In [ ]:
import matplotlib.pyplot as plt

# X and Y coordinates fetching from cartesian_data
for cartesian in cartesianList:
    #X and Y Coordinated fetching from base link
    base_line_x = cartesian[:,0]
    base_link_y = cartesian[:,1]
    plt.scatter(base_line_x,base_link_y, label ='Laser measurement')

#plotting the points
plt.title('Cartesian laser point representation expressed in the laser Reference Frame')
#Adding Title, axex names and legend and grid
plt.xlabel('Cartesian X axis')
plt.ylabel('Cartesian Y axis')
plt.grid(True)
plt.legend()
 
plt.show()

Our next task is to transform the scan data with respect to base_link and then plot the resulting points. Part of this task is to figure out the topic under which this transformation details are being published, and then use that information to perform this transformation.

Tip: In order to get the Euler angle rotation (i.e. the (roll, pitch, yaw) angles) from the transformation matrix, the method tf_transformations.euler_from_quaternion can be used.

In [ ]:
# Set the desired tf_topic_name name
tf_topic_name: str = "/tf_static"

### PLEASE EXPLAIN THE REASON TO SELECT tf_topic_name HERE ###

# YOUR CODE HERE

def getHomogeneous(trans, theta):
    t =np.array(trans)
    t =np.reshape(t, (t.shape[0], 1))

    RZ = np.array([[np.cos(theta), -np.sin(theta), 0. ],
                    [np.sin(theta), np.cos(theta), 0.],
                    [0., 0., 1.]])
    
    TZ =np.hstack((RZ, t))
    HTX =np.vstack((TZ, np.array([0, 0, 0, 1])))

    return HTX

reader.open(storage_options, converter_options)



# Set the child and parent frames between which the transformation is desired. 
child_frame: str = "base_laser_front_link"
parent_frame: str = "base_link"

# Hints: 
# * ROS messages have corresponding Python classes, so the message fields can be accessed as normal
# object field. For instance, to get linear `x' in the transformation stored 
# in tf_data_child_to_parent, `tf_data_child_to_parent.transform.linear.x' can be used.
# * You can use the same procedure as erlier to extract the transformation data from the rosbag reader.

## Create a transformation matrix using the (roll, pitch, yaw) rotation and linear (x,y,z) relation
## between the child_frame and parent_frame, and decide how to transform the above data points to
## the base_link frame of reference. Transform the Cartesian points of scan data to the base_link frame.



while reader.has_next():
    
    if count < 1:
        (topic, data, t) = reader.read_next()
        if topic == tf_topic_name:
            tf_msg = deserialize_message(data, TFMessage())
            for transform in tf_msg.transforms:
                if transform.child_frame_id == child_frame and transform.header.frame_id == parent_frame:
                    # As it is static transform, store the transform data, irrespective of the time
                    tf_data_child_to_parent = transform

                    quaternion = [tf_data_child_to_parent.transform.rotation.x,
                                  tf_data_child_to_parent.transform.rotation.y,
                                  tf_data_child_to_parent.transform.rotation.z,
                                  tf_data_child_to_parent.transform.rotation.w]
                    translation = [tf_data_child_to_parent.transform.translation.x,
                                   tf_data_child_to_parent.transform.translation.y,
                                   tf_data_child_to_parent.transform.translation.z]
                    
                    ret = tf_transformations.euler_from_quaternion(quaternion)
                    T_H_C = getHomogeneous(translation,ret[2])


cartesian_B_list = list()

for cartesian in cartesianList:
    cartesian_array = np.array(cartesian)
    homogeneous_cartesian = np.column_stack((cartesian_array, np.zeros(len(cartesian_array))))
    homogeneous_cartesian = np.column_stack((homogeneous_cartesian, np.ones(len(homogeneous_cartesian))))


    cartesian_B = np.dot(T_H_C, homogeneous_cartesian.T).T
    cartesian_B = cartesian_B[:,0:2]
    cartesian_B_list.append(cartesian_B)
cartesian_B_list

Plot the points using the Cartesian coordinates from our above implementation after transformation to base_link.

Note: Please make sure that our plot has a corresponding title, axis labels, grid, and legend.

In [ ]:
import matplotlib.pyplot as plt
 
#X and Y Coordinated fetching from base link
for cartesian_B in cartesian_B_list:
    base_line_x = cartesian_B[:,0]
    base_link_y = cartesian_B[:,1]
    plt.scatter(base_line_x,base_link_y, label ='Laser measurement')


#plotting the points
plt.title('Cartesian laser point representation expressed in the Base Reference Frame')
#Adding Title, axex names and legend and grid
plt.xlabel('Cartesian X axis')
plt.ylabel('Cartesian Y axis')
plt.grid(True)
plt.legend()
 
plt.show()

Discussions:
How does this visualisation compare to the previous visualisation of the same data? Is this what we expected?

We observed that there is 0.5m shift in the positive x-axis, which is expected when transforming the laser data from the laser link to the robots base link. Meaning, the laser scanner is mounted 0.5m forward of the robot base link. When the transform is correctly applied, the points shifted accordingly to reflect their true location relative to robots center which is required for mapping and localization.